# Distinguishing k-regular graphs

Multiple graph structures are not distinguishable by the standard message
passing Graph Neural Networks.
For instance, the Graph Neural Networks are not able to distinguish between
_k_-regular graphs of the same size.Those graphs are not isomorphic and are both $3$-regular, meaning all nodes have precisely three neighbors.

In [4]:
from neuralogic.nn import get_evaluator
from neuralogic.core import Backend
from neuralogic.core import Atom, Template, Var, Term
from neuralogic.core.settings import Settings, Optimizer
from neuralogic.utils.data import Dataset

When we assign the same features to all nodes, the messages during the
update step of message passing GNNs will be identical, resulting in
the same features and eventually classifying both graphs as the same class.
Such misclassification can be problematic in multiple domains, e.g.,
chemistry, where two indistinguishable graphs represent two different molecules.

Via the PyNeuraLogic library, we are able to embed the pattern of both graphs
or their parts. There are many alternative approaches to distinguish between
those two graphs; our presented example utilizes previously
shown encoding of triangles to capture
triangles of graph _b_, with additional rules aggregating the
general graph structure.


In [5]:
settings = Settings(optimizer=Optimizer.SGD, epochs=200)
train_dataset = Dataset()

with Template(settings).context() as template:
    template.add_rules([
        # Captures triangle
        Atom.triangle(Var.X)[1,] <= (
            Atom.edge(Var.X, Var.Y), Atom.feature(Var.Y)[1,],
            Atom.edge(Var.Y, Var.Z), Atom.feature(Var.Z)[1,],
            Atom.edge(Var.Z, Var.X), Atom.feature(Var.X)[1,],
        ),

        # Captures general graph
        Atom.general(Var.X)[1,] <= (Atom.edge(Var.X, Var.Y), Atom.feature(Var.Y)[1,]),
        Atom.general(Var.X)[1,] <= Atom.feature(Var.Y)[1,],

        Atom.predict <= Atom.general(Var.X)[1,],
        Atom.predict <= Atom.triangle(Var.X)[1,],
    ])

    train_dataset.add_example(
        [
            Atom.edge(1, 2), Atom.edge(2, 3), Atom.edge(3, 4), Atom.edge(4, 1),
            Atom.edge(2, 1), Atom.edge(3, 2), Atom.edge(4, 3), Atom.edge(1, 4),
            Atom.edge(1, 6), Atom.edge(3, 6), Atom.edge(4, 5), Atom.edge(2, 5),
            Atom.edge(6, 1), Atom.edge(6, 3), Atom.edge(5, 4), Atom.edge(5, 2),

            Atom.feature(1), Atom.feature(2), Atom.feature(3),
            Atom.feature(4), Atom.feature(5), Atom.feature(6),
        ],
    )

    train_dataset.add_example(
        [
            Atom.edge(1, 2), Atom.edge(2, 3), Atom.edge(3, 4), Atom.edge(4, 1),
            Atom.edge(2, 1), Atom.edge(3, 2), Atom.edge(4, 3), Atom.edge(1, 4),
            Atom.edge(1, 6), Atom.edge(4, 6), Atom.edge(3, 5), Atom.edge(2, 5),
            Atom.edge(6, 1), Atom.edge(6, 4), Atom.edge(5, 3), Atom.edge(5, 2),

            Atom.feature(1), Atom.feature(2), Atom.feature(3),
            Atom.feature(4), Atom.feature(5), Atom.feature(6),
        ],
    )

    train_dataset.add_queries([
        Atom.predict[1],
        Atom.predict[0],
    ])

In [6]:
neuralogic_evaluator = get_evaluator(Backend.DYNET, template)

for _ in neuralogic_evaluator.train(train_dataset):
    pass

graphs = ["a", "b"]

for graph_id, (label, predicted) in enumerate(neuralogic_evaluator.test(train_dataset)):
    print(f"Graph {graphs[graph_id]} is predicted to be class: {int(round(predicted))} | {predicted}")

Graph a is predicted to be class: 1 | 0.9096135497093201
Graph b is predicted to be class: 0 | 0.03949236869812012
